In [1]:
from parse_documents import *

# filepath = 'upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg'  # Replace with the path of your file
# filepath = "upload/World_Energy_By_Country_And_Region_1965_to_2023.csv"
# filepath = 'upload/Diabetic-retinopathy-identification-using-parallel-convo_2023_Expert-Systems.pdf'
filepath = 'upload/bitcoin.pdf'

file_metadata_json = parse_document(filepath)

# metadata = parse_document.get('metadata', {})

raw_original_file_name = parsed_document['resourceName']
ic(raw_original_file_name)



<class 'dict'>
dict_keys(['metadata', 'content', 'status'])
{'pdf:unmappedUnicodeCharsPerPage': ['0', '0', '0', '0', '0', '0', '0', '0', '0'], 'pdf:PDFVersion': '1.4', 'xmp:CreatorTool': 'Writer', 'pdf:hasXFA': 'false', 'access_permission:modify_annotations': 'true', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dcterms:created': '2009-03-24T17:33:15Z', 'dc:format': 'application/pdf; version=1.4', 'pdf:docinfo:creator_tool': 'Writer', 'pdf:overallPercentageUnmappedUnicodeChars': '0.0', 'access_permission:fill_in_form': 'true', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'pdf:containsNonEmbeddedFont': 'false', 'Content-Length': '184292', 'pdf:hasMarkedContent': 'false', 'Content-Type': 'application/pdf', 'dc:language': 'en-GB', 'pdf:producer': 'OpenOffice.org 2.4', 'pdf:totalUnmappedUn

NameError: name 'parsed_document' is not defined

In [ ]:
from tika import parser

# Open your file in binary mode to get its raw bytes
with open(filepath, 'rb') as f:
    content = f.read()

parser = parser.from_bytes(content, metadata=True)

# The 'filename' field of parser.metadata should now contain the original name with preserved encoding.
print(parser.metadata['filename'])

In [ ]:
import chardet

# Open your file in binary mode to get its raw bytes
with open(filepath, 'rb') as f:
    content = f.read()

parser = tika.parser.from_bytes(content, metadata=True)
encoding_guess = chardet.detect(parser.get_content())

if encoding_guess['confidence'] > 0.9:  # Confidence threshold can be adjusted based on your needs
    # Use the detected encoding to re-interpret the file name correctly.
    original_filename = parser.metadata['resourceName'].decode(encoding_guess['encoding'])
else:
    # For cases where detection is uncertain or fails, you might consider falling back to UTF-8 (or another default).
    original_filename = parser.metadata['resourceName'].decode('utf-8', errors='replace')

print(original_filename)

In [ ]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Print the type and keys of the parsed document
        print(type(parsed_document))
        print(parsed_document.keys())

        # Extract and print metadata if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            print("Metadata:")
            print(metadata)
        else:
            print("No metadata found.")
        
        # Print content if needed (commented to avoid large outputs)
        # print(parsed_document.get('content', 'No content available'))
        
    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


In [ ]:
from tika import parser

def parse_document(filepath):
    try:
        parsed_document = parser.from_file(filepath)
        
        # Check if parsing was successful
        if 'status' in parsed_document and parsed_document['status'] == 200:
            print(f"Parsing successful: {parsed_document['status']}")
        else:
            print(f"Parsing failed with status: {parsed_document.get('status')}")
            return
        
        # Extract and print list of keys in the parsed document
        document_keys = list(parsed_document.keys())
        print("Keys in the parsed document:")
        print(document_keys)

        # Extract metadata keys if available
        metadata = parsed_document.get('metadata', {})
        if metadata:
            metadata_keys = list(metadata.keys())
            print("Keys in metadata:")
            print(metadata_keys)
        else:
            print("No metadata found.")

    except FileNotFoundError:
        print(f"Error: File '{filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
parse_document("upload/Jean-Baptiste_Perronneau_-_Magdaleine_Pinceloup_de_la_Grange,_née_de_Parseval.jpg")


In [ ]:
import os
import json
from tika import parser
from ipfs_functions import *

def parse_documents_in_directory(directory_path):
    temp_dir = "tmp" # Get the system's temporary directory
    print(f"Saving metadata files in temporary directory: {temp_dir}")

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        upload_file_to_ipfs(directory_path + "/" + filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            try:
                parsed_document = parser.from_file(file_path)
                
                # Check if parsing was successful
                if 'status' in parsed_document and parsed_document['status'] == 200:
                    metadata = parsed_document.get('metadata', {})
                    print(metadata)
                    upload_json_to_ipfs(metadata)
                    
                    # Save metadata as JSON
                    if metadata:
                        json_filename = f"{os.path.splitext(filename)[0]}.json"
                        json_path = os.path.join(temp_dir, json_filename)

                        with open(json_path, 'w') as json_file:
                            json.dump(metadata, json_file, indent=4)
                        
                        print(f"Metadata saved for '{filename}' as '{json_filename}'")
                    else:
                        print(f"No metadata found for '{filename}'")
                else:
                    print(f"Parsing failed for '{filename}' with status: {parsed_document.get('status')}")
            
            except Exception as e:
                print(f"An error occurred with file '{filename}': {e}")

# Example usage
parse_documents_in_directory("upload")


In [ ]:
import os
import json
from tika import parser
from ipfs_functions import *

def parse_documents_in_directory(directory_path):
    

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        print(file_path)
        upload_file_to_ipfs(file_path)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            try:
                parsed_document = parser.from_file(file_path)
                
                # Check if parsing was successful
                if 'status' in parsed_document and parsed_document['status'] == 200:
                    metadata = parsed_document.get('metadata', {})
                    upload_json_to_ipfs(metadata)
                    
                else:
                    print(f"Parsing failed for '{filename}' with status: {parsed_document.get('status')}")
            
            except Exception as e:
                print(f"An error occurred with file '{filename}': {e}")

# Example usage
parse_documents_in_directory("upload")
